## Libraries

In [18]:
import requests
import json

In [19]:
#data manipulation + import
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import json
import http.client

In [20]:
#dealing with time
import datetime
from datetime import datetime

## Import branches through open banking API

# CHANGE THE PERIOD

In [21]:
period = '2024_05'
bank = 'Barclays'

In [22]:
#definition of output file path

input_file_json = '../../../output/'+ period + '_Branches/' + bank + '_'+ period +'.json'

output_file_json = '../../../output/'+ period + '_Branches/' + bank + '_'+ period +'.json'

input_file_csv = '../../../output/'+ period + '_Branches/'+ bank + '_'+ period + '.csv'

output_file_csv = '../../../output/'+ period + '_Branches/'+ bank + '_'+ period + '_capacity.csv'


Calling the barclays API

In [6]:
url = "https://atlas.api.barclays:443/open-banking/v2.2/branches"
headers = { "cache-control" : "no cache"}

#https://atlas.api.barclays:443/open-banking/v2.2

In [7]:
#params = {"positions":[0,6,7,29]}

# Make a get request with the parameters.
response = requests.get(url, headers=headers)

# Print the content of the response
#print(response.content)

In [8]:
data = response.json()

The resonse has two parts:

metadata (data['metadata']
actual data (data['data']
Here we display the content of the metadata. This includes information about the last time the data was updated on the backend, the number of results, etc:

data['meta']
data['meta']

In [9]:
data['meta']

{'LastUpdated': '2024-05-01T14:30:06+0100',
 'TotalResults': 726,
 'Agreement': 'Use of the APIs and any related data will be subject to the terms of the Open Licence and subject to terms and conditions',
 'License': 'https://www.openbanking.org.uk/open-licence',
 'TermsOfUse': 'https://www.openbanking.org.uk/terms'}

In [10]:
with open(output_file_json, 'w') as f:
    json.dump(data, f)

In [11]:
with open(input_file_json, "r") as read_file:
    data = json.load(read_file)

In [12]:
data

{'meta': {'LastUpdated': '2024-05-01T14:30:06+0100',
  'TotalResults': 726,
  'Agreement': 'Use of the APIs and any related data will be subject to the terms of the Open Licence and subject to terms and conditions',
  'License': 'https://www.openbanking.org.uk/open-licence',
  'TermsOfUse': 'https://www.openbanking.org.uk/terms'},
 'data': [{'Brand': [{'BrandName': 'Barclays UK',
     'Branch': [{'Identification': 'UK-B-20031804',
       'SequenceNumber': '0',
       'Name': 'Barclays Bank',
       'Type': 'Physical',
       'Photo': 'http://www.barclays.co.uk/branchImages/content/dam/branchfinder/flex-images/Local Generic Image.jpg',
       'CustomerSegment': ['Private',
        'SME',
        'Premier',
        'Select',
        'Wealth',
        'Corporate',
        'Business',
        'Personal'],
       'Accessibility': ['WheelchairAccess', 'InductionLoop'],
       'Availability': {'StandardAvailability': {'Day': [{'Name': 'Monday',
           'OpeningHours': [{'OpeningTime': '00:

In [13]:
data.keys()

dict_keys(['meta', 'data'])

In [14]:
data['data'][0]['Brand']

[{'BrandName': 'Barclays UK',
  'Branch': [{'Identification': 'UK-B-20031804',
    'SequenceNumber': '0',
    'Name': 'Barclays Bank',
    'Type': 'Physical',
    'Photo': 'http://www.barclays.co.uk/branchImages/content/dam/branchfinder/flex-images/Local Generic Image.jpg',
    'CustomerSegment': ['Private',
     'SME',
     'Premier',
     'Select',
     'Wealth',
     'Corporate',
     'Business',
     'Personal'],
    'Accessibility': ['WheelchairAccess', 'InductionLoop'],
    'Availability': {'StandardAvailability': {'Day': [{'Name': 'Monday',
        'OpeningHours': [{'OpeningTime': '00:00', 'ClosingTime': '00:00'}]},
       {'Name': 'Tuesday',
        'OpeningHours': [{'OpeningTime': '09:30', 'ClosingTime': '16:30'}]},
       {'Name': 'Wednesday',
        'OpeningHours': [{'OpeningTime': '00:00', 'ClosingTime': '00:00'}]},
       {'Name': 'Thursday',
        'OpeningHours': [{'OpeningTime': '09:30', 'ClosingTime': '16:30'}]},
       {'Name': 'Friday',
        'OpeningHours': [{'O

In [15]:
#data['data'][0].keys

In [16]:
#data['src/main/resources/data'][0].keys()

In [17]:
f = open(input_file_csv, "w")

f.write('Brand,ID,Type,Branch Name,Town,Postcode,Lat,Long,MonOp,MonCls,TueOp,TueCls,WedOp,WedCls,ThuOp,ThuCls,FriOp,FriCls,SatOp,SatCls,SunOp,SunCls\n')

for brand in data['data'][0]['Brand']:
    for branch in brand['Branch']:
        ID = branch['Identification']
        Btype =  branch ['Type']
        BName = branch['Name']
        town = branch ['PostalAddress'].get('TownName', 'None')
        postcode = branch['PostalAddress']['PostCode']
        try:
            latitude = float(branch ['PostalAddress']['GeoLocation']['GeographicCoordinates']['Latitude'])
            longitude = float(branch['PostalAddress']['GeoLocation']['GeographicCoordinates']['Longitude'])
        except KeyError:
            latitude = 0
            longitude = 0
            
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        opening = {day: {'Open':'00:00', 'Close': '00:00'} for day in days}
        try:
            availability = branch['Availability']['StandardAvailability']['Day']
            for day in availability:
                opening[day['Name']]['Open'] = day['OpeningHours'][0]['OpeningTime']
                opening[day['Name']]['Close'] = day['OpeningHours'][0]['ClosingTime']
        except KeyError:
            pass
        f.write('{0},{1},{2},{3},{4},{5},{6:6f},{7:6f}'.format(brand['BrandName'], ID, Btype, BName, town, postcode, latitude, longitude))
        for day in days:
            f.write(',{0},{1}'.format(opening[day]['Open'],opening[day]['Close']))
        f.write('\n')
f.close()

In [18]:
# CHECK THE CSV FILE & cORRECTS before reunning this bc it gives many errors

In [23]:
branches = pd.read_csv(input_file_csv)

In [24]:
branches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Brand        726 non-null    object 
 1   ID           726 non-null    object 
 2   Type         715 non-null    object 
 3   Branch Name  726 non-null    object 
 4   Town         726 non-null    object 
 5   Postcode     726 non-null    object 
 6   Lat          726 non-null    float64
 7   Long         726 non-null    float64
 8   MonOp        726 non-null    object 
 9   MonCls       726 non-null    object 
 10  TueOp        726 non-null    object 
 11  TueCls       726 non-null    object 
 12  WedOp        726 non-null    object 
 13  WedCls       726 non-null    object 
 14  ThuOp        726 non-null    object 
 15  ThuCls       726 non-null    object 
 16  FriOp        726 non-null    object 
 17  FriCls       726 non-null    object 
 18  SatOp        726 non-null    object 
 19  SatCls  

In [25]:
#calculate total open time (in minutes), on a Monday
branches['MonDuration']=(pd.to_datetime(branches['MonCls']) - pd.to_datetime(branches['MonOp']))/np.timedelta64(1,'m')

In [26]:
#calculate total open time (in minutes), on a Tuesday
branches['TueDuration']=(pd.to_datetime(branches['TueCls']) - pd.to_datetime(branches['TueOp']))/np.timedelta64(1,'m')

In [27]:
#calculate total open time (in minutes), on a Wednesday
branches['WedDuration']=(pd.to_datetime(branches['WedCls']) - pd.to_datetime(branches['WedOp']))/np.timedelta64(1,'m')

In [28]:
#calculate total open time (in minutes), on a Thursday
branches['ThuDuration']=(pd.to_datetime(branches['ThuCls']) - pd.to_datetime(branches['ThuOp']))/np.timedelta64(1,'m')

In [29]:
#calculate total open time (in minutes), on a Friday
branches['FriDuration']=(pd.to_datetime(branches['FriCls']) - pd.to_datetime(branches['FriOp']))/np.timedelta64(1,'m')

In [30]:
#calculate total open time (in minutes), on a Saturday
branches['SatDuration']=(pd.to_datetime(branches['SatCls']) - pd.to_datetime(branches['SatOp']))/np.timedelta64(1,'m')

In [31]:
#calculate total open time (in minutes), on a Sunday
branches['SunDuration']=(pd.to_datetime(branches['SunCls']) - pd.to_datetime(branches['SunOp']))/np.timedelta64(1,'m')

In [32]:
branches['TotalWkOp']= branches['MonDuration']+branches['TueDuration']+ \
                        branches['WedDuration']+branches['ThuDuration']+ \
                        branches['FriDuration']+branches['SatDuration']+branches['SunDuration']

In [33]:
branches.head()

,Brand,ID,Type,Branch Name,Town,Postcode,Lat,Long,MonOp,MonCls,...,SunOp,SunCls,MonDuration,TueDuration,WedDuration,ThuDuration,FriDuration,SatDuration,SunDuration,TotalWkOp
0,Barclays UK,66000114,Physical,Barclays Bank,Wisbech,PE13 1EU,52.664243,0.161065,00:00,00:00,...,00:00,00:00,0.0,420.0,0.0,0.0,420.0,0.0,0.0,840.0
1,Barclays UK,66000101,Physical,Barclays Bank,Guisborough,TS14 6AP,54.535211,-1.056695,00:00,00:00,...,00:00,00:00,0.0,300.0,0.0,300.0,0.0,0.0,0.0,600.0
2,Barclays UK,UK-B-20252909,Physical,Barclays Bank,9-11 Chapel Row,DL12 0SN,54.623238,-2.081710,00:00,00:00,...,00:00,00:00,0.0,480.0,0.0,0.0,480.0,0.0,0.0,960.0
3,Barclays UK,66000099,Physical,Barclays Bank,Chorley,PR7 1AL,53.654856,-2.629535,09:30,16:30,...,00:00,00:00,420.0,420.0,0.0,0.0,420.0,0.0,0.0,1260.0
4,Barclays UK,66000123,Physical,Barclays Bank,Melton Mowbray,LE13 1XD,52.763622,-0.885957,00:00,00:00,...,00:00,00:00,0.0,300.0,300.0,0.0,300.0,0.0,0.0,900.0


In [34]:
# The capacity of a branch is defined as the total nr of minutes open per week divided by 2790mins or 46.5h 
# whihc is the typical opening schedule of a physical branch
branches['Capacity']= branches['TotalWkOp']/2790

In [35]:
branches.to_csv(output_file_csv)

In [27]:
px.scatter(
    branches, 
    x="Capacity", 
    y="Lat", 
    color='Type',
    title='Barclays Mobile and Physical Branches Capacity by Geographical Latitude, April 2020')

NameError: name 'px' is not defined

In [ ]:
px.scatter(branches, x="Long", y="Lat", color="Capacity", facet_col="Type",
           color_continuous_scale=px.colors.sequential.Viridis, render_mode="webgl")

In [ ]:
px.set_mapbox_access_token('pk.eyJ1IjoiYW5kcmFzb25lYSIsImEiOiJjam9paTNhaW0wMnU5M3FzMXQwY2FmNmYwIn0.epbBdAlbR55LPW7UF6Gt0w')

In [ ]:
px.scatter_mapbox(branches, lat="Lat", lon="Long", color="Type", size="Capacity", hover_name="Branch Name",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=20, zoom=7, title = "Barclays Physical and Mobile Branches, Nov 2019")